In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Self comments:
#We will import all the required packages in this cell
import torch
import torchaudio
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn 
from sklearn import model_selection
from torchinfo import summary
from tqdm import tqdm
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('/kaggle/input/urbansound8k/UrbanSound8K.csv')
# df.head(5)

In [4]:
wave = torchaudio.load('/kaggle/input/urbansound8k/fold4/107653-9-0-18.wav')
# plt.plot(wave[0].t().numpy())
print(wave[0].shape)

torch.Size([2, 176400])


In [5]:
class MyDataset:
    def __init__(self,audio_csv,Id):          
        self.audio_csv = audio_csv          
        self.Id = Id       
        
    def __len__(self):
        return len(self.audio_csv)
    
    def __getitem__(self,index):
        path = self.audio_csv[index]
        wave, sampling_rate = torchaudio.load(path)
        mono_audio = torch.mean(wave, dim=0, keepdim=True)
        tempData = torch.zeros([1,160000])
        if mono_audio.numel() < 160000:
            tempData[:,:mono_audio.numel()] = mono_audio
        else:
            tempData[:,:160000]
        mono_audio = tempData
        
        mfcc = torchaudio.transforms.MFCC(sample_rate=sampling_rate)(mono_audio) #channels,n_mffcs,time
        mfcc_norm = (mfcc - mfcc.mean())/ (mfcc.std())
        return {
            "mfcc" : torch.tensor(mfcc_norm, dtype=torch.float),
            "label" : torch.tensor(self.Id[index], dtype=torch.long)
        }

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
def collate_fn(data):
    mfccs = []
    labels = []
    for d in data:
        mfcc = d["mfcc"].to(device)
        label = d["label"].to(device)
        mfccs.append(mfcc)
        labels.append(label)
    
    mfcc = torch.nn.utils.rnn.pad_sequence(mfccs, batch_first = True, padding_value=0.0)
    labels = torch.tensor(labels)
#     print(mfcc.shape)  [128,1,40,801] batch,channel,time,value
    return mfcc, labels

In [8]:
# /kaggle/input/urbansound8k/fold4/107653-9-0-12.wav
FILE_PATH = "/kaggle/input/urbansound8k/"

if __name__ == "__main__":
    df = pd.read_csv("/kaggle/input/urbansound8k/UrbanSound8K.csv")
    files = df["slice_file_name"].values.tolist()
    folder_fold = df["fold"].values
    label = df["classID"].values.tolist()
    path = [
        os.path.join(FILE_PATH + "fold" + str(folder) + "/" + file) for folder, file in zip(folder_fold, files)
    ]
    X_train, X_test, y_train, y_test = model_selection.train_test_split(path, label, random_state=42, test_size=0.2)
    
    train_dataset = MyDataset(
        audio_csv=X_train,
        Id=y_train
    )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=64, shuffle=True, drop_last=True, collate_fn=collate_fn
    )
    
    test_dataset = MyDataset(
        audio_csv=X_test,
        Id=y_test
    )
    
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=64, shuffle=False, drop_last=True, collate_fn=collate_fn
    )

In [9]:
class ModelLSTM(nn.Module):
    def __init__(self, n_feature=5, out_feature=5, n_hidden=256, n_layers=2, drop_prob=0.3):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_feature = n_feature
        self.lstm = nn.LSTM(self.n_feature, self.n_hidden, self.n_layers, dropout= self.drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc1 = nn.Linear(int(n_hidden), int(n_hidden/2))
        self.bn1 = nn.BatchNorm1d(int(n_hidden/2))
        self.fc2 = nn.Linear(int(n_hidden/2), out_feature)
        self.relu = nn.ReLU()
        
    def forward(self, x, hidden):
#         print(x.shape)
        x=torch.squeeze(x,1)
#         print(x.shape)
        l_out, l_hidden = self.lstm(x, hidden)
        out = self.fc1(l_out[:, -1, :])
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)

        
        return out, l_hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        return hidden

In [10]:
summary(ModelLSTM())


Layer (type:depth-idx)                   Param #
ModelLSTM                                --
├─LSTM: 1-1                              795,648
├─Dropout: 1-2                           --
├─Linear: 1-3                            32,896
├─BatchNorm1d: 1-4                       256
├─Linear: 1-5                            645
├─ReLU: 1-6                              --
Total params: 829,445
Trainable params: 829,445
Non-trainable params: 0

In [11]:
def train(data_loader, model, epoch, optimizer, device):
    losses = []
    accuracies = []
    labels = []
    preds = []
    model.train()
    loop = tqdm(data_loader)
    for batch_idx, (data, target) in enumerate(loop):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        output, hidden_state = model(data, model.init_hidden(64))
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        probs = torch.softmax(output, dim=1)
        winners = probs.argmax(dim=1)
        corrects = (winners == target)
        accuracy = corrects.sum().float() / float(target.size(0))
        accuracies.append(accuracy)
        labels += torch.flatten(target).cpu()
        preds += torch.flatten(winners).cpu()
        loop.set_description(f"EPOCH: {epoch+1} | ITERATION : {batch_idx}/{len(data_loader)} | LOSS: {loss.item()}| ACCURACY: {accuracy}")
        loop.set_postfix(loss=loss.item())
    
    avg_train_loss = sum(losses) / len(losses)
    avg_train_accuracy = (sum(accuracies) / len(accuracies))*100
    print(f"Average Train Loss: {avg_train_loss:.4f}, Average Train Accuracy: {avg_train_accuracy:.2f}%")
    return avg_train_loss, avg_train_accuracy

In [12]:
def test(data_loader, model, device):
    model.eval()
    accs = []
    preds = []
    labels = []
    test_accuracies = []
    with torch.no_grad():
        loop = tqdm(data_loader) 
        for batch_idx, (data, target) in enumerate(loop):
            data = data.to(device)
            target = target.to(device)
            output, hidden_state = model(data, model.init_hidden(64))
            probs = torch.softmax(output, dim=1)
            winners = probs.argmax(dim=1)
            corrects = (winners == target)
            accuracy = corrects.sum().float() / float(target.size(0))
            
            test_accuracies.append(accuracy)
            labels += torch.flatten(target).cpu()
            preds += torch.flatten(winners).cpu()
            
    avg_test_acc = (sum(test_accuracies) / len(test_accuracies))*100
    print(f"Average Test Accuracy: {avg_test_acc:.2f}%")

    return avg_test_acc

In [13]:
EPOCH = 50
OUT_FEATURE = 10
SAVE_PATH = "/kaggle/working/"

In [14]:
def main():
    
    model = ModelLSTM(n_feature=801, out_feature = OUT_FEATURE).to(device)
    optimizer = torch.optim.AdamW(model.parameters())

    for epoch in range(EPOCH):
        avg_train_loss, avg_train_acc = train(train_loader, model, epoch, optimizer, device)
        avg_test_acc = test(test_loader, model, device)
        filename = f"{SAVE_PATH}ep-{epoch+1}-loss-{avg_train_loss:.3f}-acc-{avg_train_acc:.2f}.pth"
        torch.save(model.state_dict(), filename)
        print(f"Model saved as {filename}")
        print('-------------------------------------------')
    print('Finished Training')

if __name__ == "__main__":
    main()

  0%|          | 0/109 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(
/tmp/ipykernel_23/1068264031.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "mfcc" : torch.tensor(mfcc_norm, dtype=torch.float),
EPOCH: 1 | ITERATION : 108/109 | LOSS: 2.216891288757324| ACCURACY: 0.15625: 100%|██████████| 109/109 [03:35<00:00,  1.98s/it, loss=2.22]


Average Train Loss: 2.2582, Average Train Accuracy: 13.33%


100%|██████████| 27/27 [00:57<00:00,  2.12s/it]


Average Test Accuracy: 10.42%
Model saved as /kaggle/working/ep-1-loss-2.258-acc-13.33.pth
-------------------------------------------


EPOCH: 2 | ITERATION : 108/109 | LOSS: 2.1915013790130615| ACCURACY: 0.140625: 100%|██████████| 109/109 [02:02<00:00,  1.12s/it, loss=2.19]


...

Average Test Accuracy: 25.64%
Model saved as /kaggle/working/ep-49-loss-1.805-acc-28.27.pth
-------------------------------------------


EPOCH: 50 | ITERATION : 108/109 | LOSS: 1.9126222133636475| ACCURACY: 0.234375: 100%|██████████| 109/109 [02:03<00:00,  1.13s/it, loss=1.91]


Average Train Loss: 1.8072, Average Train Accuracy: 27.75%


100%|██████████| 27/27 [00:31<00:00,  1.16s/it]

Average Test Accuracy: 22.92%
Model saved as /kaggle/working/ep-50-loss-1.807-acc-27.75.pth
-------------------------------------------
Finished Training
